In [12]:
# #Add the appropriate module
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.optimizers import Adam
import pandas as pd 
import deepchem as dc
import numpy as np
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
import matplotlib.pyplot as plt

In [7]:
# # Generate data sets.
polymer_data = pd.read_csv('E:/Training files/Image-CNN/data.csv',encoding= "unicode_escape")
polymer_info = polymer_data.values[:]

index_list = []
PID_list = []
tg_list = []
smile_list = []
for i in range(len(polymer_info)):
    temp0,temp1,temp2,temp3 = polymer_info[i]
    index_list.append(temp0)
    PID_list.append(temp1)
    tg_list.append(temp2)
    smile_list.append(temp3)

idx_train = []
with open('E:/Training files/Image-CNN/dataset_test.txt') as f :
    for i in f:
        idx_train.append(int(i))
        
idx_valid = []
with open('E:/Training files/Image-CNN/dataset_valid.txt') as f :
    for i in f:
        idx_valid.append(int(i))
    
idx_test = []
with open('E:/Training files/Image-CNN/dataset_test.txt') as f :
    for i in f:
        idx_test.append(int(i))
        
encodings_list = []
targets_list = []
featurizer = dc.feat.OneHotFeaturizer(charset = ['*', 'c', '1', '(', 'C', '2', 'N', '3', '=', 'O', ')',
                                                 '4', '5', '6', '7', '8', 'F', 'n', 'S', '-', 's', '[', 
                                                 '@', 'H', ']', 'i', 'o', '9', '%', '0', 'P', '+', '#', 
                                                 'l', 'B', 'r'],max_length = 250)
for num in idx_train:
    temp=[]
    smiles = smile_list[num]
    temp.append(smiles) 
    encodings = featurizer.featurize(temp)
    a,b,c = encodings.shape
    encodings = encodings.reshape((b,c,1))
    encodings_list.append(encodings)
    target = tg_list[num]
    targets_list.append(target)

train_X = np.array(encodings_list)
train_Y =  np.array(targets_list)

 
encodings_list = []
targets_list = []  
for num in idx_valid:
    temp=[]
    smiles = smile_list[num]
    temp.append(smiles) 
    encodings = featurizer.featurize(temp)
    a,b,c = encodings.shape
    encodings = encodings.reshape((b,c,1))
    encodings_list.append(encodings)
    target = tg_list[num]
    targets_list.append(target)
    
Valid_X = np.array(encodings_list)
Valid_Y =  np.array(targets_list)

encodings_list = []
targets_list = []     
for num in idx_test:
    temp=[]
    smiles = smile_list[num]
    temp.append(smiles) 
    encodings = featurizer.featurize(temp)
    a,b,c = encodings.shape
    encodings = encodings.reshape((b,c,1))
    encodings_list.append(encodings)
    target = tg_list[num]
    targets_list.append(target)
    
Test_X = np.array(encodings_list)
Test_Y =  np.array(targets_list)

In [15]:
# # Train model
img_width, img_height = 250, 37
X_train = train_X.astype('float32') 
X_valid = Valid_X.astype('float32') 
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, epsilon=1e-4, mode='min',min_lr=0.00001)
mcp_save = ModelCheckpoint('image-cnn-34.hdf5', save_best_only=True, monitor='val_loss', mode='min')
adam = Adam(lr=0.001)

model = Sequential()
model.add(Conv2D(16, (8, 8), activation='relu',
                 input_shape=(img_width, img_height, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(6, (4, 4), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1536, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu')) 
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer=adam, metrics=['mean_absolute_error'])
history = model.fit(X_train, train_Y,  validation_data=(X_valid, Valid_Y),
          epochs=100, 
          batch_size = 4 ,
          callbacks=[reduce_lr_loss,mcp_save])

Epoch 1/100


D:\Anaconda3\envs\deepchem_1\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


182/182 [==============================] - 2s 9ms/step - loss: 11041.6855 - mean_absolute_error: 82.1215 - val_loss: 7594.3047 - val_mean_absolute_error: 68.3303 - lr: 0.0010
Epoch 2/100
182/182 [==============================] - 1s 8ms/step - loss: 7784.4482 - mean_absolute_error: 70.7729 - val_loss: 6672.1689 - val_mean_absolute_error: 64.1105 - lr: 0.0010
Epoch 3/100
182/182 [==============================] - 2s 8ms/step - loss: 6007.3398 - mean_absolute_error: 62.2287 - val_loss: 5249.1694 - val_mean_absolute_error: 58.0390 - lr: 0.0010
Epoch 4/100
182/182 [==============================] - 1s 8ms/step - loss: 4686.3867 - mean_absolute_error: 53.4545 - val_loss: 4144.4814 - val_mean_absolute_error: 50.6917 - lr: 0.0010
Epoch 5/100
182/182 [==============================] - 2s 9ms/step - loss: 3864.7810 - mean_absolute_error: 48.3783 - val_loss: 3690.3252 - val_mean_absolute_error: 46.8865 - lr: 0.0010
Epoch 6/100
182/182 [==============================] - 1s 8ms/step - loss: 3404.8

In [17]:
#Predict

load_model = load_model("image-cnn-34.hdf5")
np.set_printoptions(precision=4)
predicted_tr = load_model.predict(train_X)
predicted_va = load_model.predict(Valid_X)
predicted_te = load_model.predict(Test_X)

23/23 [==============================] - 0s 2ms/step


In [19]:
train_Y[0],predicted_tr[0]

(258.0, array([302.7223], dtype=float32))